# Segmenting satellite imagery from the Maxar Open Data Program

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/maxar_open_data.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/maxar_open_data.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/maxar_open_data.ipynb)

This notebook shows how to segment satellite imagery from the [Maxar Open Data](https://github.com/opengeos/maxar-open-data) program for Libya floods.  

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [1]:
%pip install segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 732.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import leafmap
from samgeo import SamGeo, raster_to_vector, overlay_images

## Download sample data

First, let's download a sample image of Derna, Libya from [here](https://drive.google.com/file/d/1jIIC5hvSPeJEC0fbDhtxVWk2XV9AxsQD/view?usp=sharing).

In [4]:
url = "https://github.com/opengeos/datasets/releases/download/raster/Derna_sample.tif"

In [5]:
leafmap.download_file(url, output="image.tif")

Downloading...
From: https://github.com/opengeos/datasets/releases/download/raster/Derna_sample.tif
To: /content/image.tif
100%|██████████| 12.2M/12.2M [00:00<00:00, 28.2MB/s]


'/content/image.tif'

## Create an interactive map

In [ ]:
m = leafmap.Map(height="600px")
m.add_basemap("SATELLITE")
m.add_raster("image.tif", layer_name="Image")
m.add_layer_manager()
m

## Initialize SAM class

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [7]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 80,
}

In [8]:
sam = SamGeo(
    model_type="vit_h",
    sam_kwargs=sam_kwargs,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:35<00:00, 71.8MB/s]


## Segment the image

In [ ]:
sam.generate("image.tif", output="mask.tif", foreground=True)

## Convert raster to vector

In [ ]:
raster_to_vector("mask.tif", output="mask.shp")

## Display the segmentation result

First, let's show the result as a binary image.

In [ ]:
sam.show_masks(cmap="binary_r")

Display the annotations (each mask with a random color).

In [ ]:
sam.show_anns(axis="off", opacity=1, output="annotation.tif")

## Compare images with a slider

In [ ]:
leafmap.image_comparison(
    "image.tif",
    "annotation.tif",
    label1="Image",
    label2="Segmentation",
)

Overlay the annotations on the image and use the slider to change the opacity interactively.

In [ ]:
overlay_images("image.tif", "annotation.tif", backend="TkAgg")

## Display images on an interactive map.

In [ ]:
m.add_raster("mask.tif", layer_name="Mask", nodata=0)
m.add_raster("annotation.tif", layer_name="Annotation")
m

In [ ]:
m.add_vector("mask.shp", layer_name="Vector", info_mode=None)

![](https://i.imgur.com/kjMi8AF.png)